<a href="https://colab.research.google.com/github/KIMDOKYOUNG/DeepLearning/blob/master/chapter7/chapter_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 밑바닥부터 시작하는 딥러닝 1 뽀개기
#### CHAPTER 7 합성곱 신경망(CNN)
#### 일자 : 2020-08-22


## Convolutional Neural Network
- 합성곱 계층(Conv)과 풀링 계층(Pooling)이 추가된다.
- [Conv - ReLU - Pooling] - [Conv - ReLU - Pooling] - [Conv - ReLU - Pooling] - [Affine - ReLU] - [Affine -Softmax]
- padding
- stride
- 이미지는 3차원 형상이며, 담겨있는 공간적 정보를 사용할 수 있게 된다. 

### 합성곱 연산 
- 필터 연산과 같다.
- 필터의 크기 = 윈도우(window)사이즈
- stride만큼 이동하며 입력과 필터에 대응한느 원소끼리 곱한 후 그 총합을 구하여 해당 위치에 저장한다. 그것이 feature map이 된다.
- bais는 원소에 각각 더해진다.

### Padding
- 입럭 데이터 주변을 특정값으로 채운다. 
- 합성곱 연산후 출력크기가 줄어드는 것을 방지하기 위해서 

### Stride
- 필터를 적용하는 위치의 간격
- 필터 윈도우가 stride만큼 움직인다. 

### 출력크기 계산하기

- 입력크기 : H, W
- 필터 크기 : FH,FW
- 출력 크기 : OH, OW
- 패딩 : P
- 스트라이드 : S

$$ OH = \frac{H+2P-FH}{S} + 1$$

$$ OW = \frac{W+2P-FW}{S} + 1$$

### 풀링 계층
- 세로, 가로 방향의 공간을 줄이는 연산 
- max pooling

---

### 합성곱/ 풀링 계층 구현하기 



In [1]:
import numpy as np

x = np.random.rand(10,1,28,28)
x.shape


(10, 1, 28, 28)

In [5]:
print(x[0].shape)
print(x[1].shape)
print(x[0][0].shape)

(1, 28, 28)
(1, 28, 28)
(28, 28)


- im2col로 데이터를 1줄로 펼친다. 
- 필터를 세로로 1열로 전개하고, im2col이 전개한 데이터와 행렬 내적을 계산하고 출력 데이터(2차원)을 reshape한다. 

In [6]:
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    """다수의 이미지를 입력받아 2차원 배열로 변환한다(평탄화).
    
    Parameters
    ----------
    input_data : 4차원 배열 형태의 입력 데이터(이미지 수, 채널 수, 높이, 너비)
    filter_h : 필터의 높이
    filter_w : 필터의 너비
    stride : 스트라이드
    pad : 패딩
    
    Returns
    -------
    col : 2차원 배열
    """
    N, C, H, W = input_data.shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1

    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    return col

In [7]:
x1 = np.random.rand(1,3,7,7) # (데이터 수 , 채널 수, 높이, 너비)
col1 = im2col(x1,5,5,stride=1, pad = 0)
print(col1.shape)

(9, 75)


In [8]:
x2 = np.random.rand(10,3,7,7) # (데이터 수 , 채널 수, 높이, 너비)
col2 = im2col(x2,5,5,stride=1, pad = 0)
print(col2.shape)

(90, 75)


In [10]:
class Convolution:
  def __init__(self,W,b,stride = 1, pad = 0):
    self.W = W
    self.b = b
    self.stride = stride
    self.pad = pad

  def forward(self,x):
    FN, C, FH, FW = self.W.shape
    N, C, H, W = x.shape
    out_h = int(1 + (H + 2*self.pad - FH)/ self.stride)
    out_w = int(1 + (W + 2*self.pad - FW)/ self.stride)

    col = im2col(x, FH, FW, self.stride, self.pad)
    col_W = self.W.reshape(FN, -1).T
    out = np.dot(col,col_W) + self.b

    out = out.reshape(N,out_h, out_w, -1).transpose(0,3,1,2)

    return out 

In [11]:
class Pooling:
  def __init__(self,pool_h, pool_w, stride = 1, pad = 0):
    self.pool_h = pool_h
    self.pool_w = pool_w
    self.stride = stride
    self.pad = pad

  def forward(self,x):
    N, C, H, W = x.shape
    out_h = int(1 + (H - self.pool_h)/ self.stride)
    out_w = int(1 + (W - self.pool_w)/ self.stride)
    # 입력데이터 전개 
    col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
    col = col.reshape(-1, self.pool_h*self.pool_w)
    # 최댓값
    out = np.max(col, axis = 1)
    # reshape
    out = out.reshape(N,out_h, out_w, C).transpose(0,3,1,2)

    return out 

### CNN 구현하기 

In [13]:
class SimpleConvNet:
  def __init__(self,input_dim = (1,28,28), conv_param = {'filter_num':30,'filter_size':5, 'pad':0,'stride':1}, 
               hidden_size = 100, output_size = 10, weight_init_std = 0.01):
    fileter_num = conv_param['filter_num']
    filter_size = conv_param['filter_size']
    filter_pad = conv_param['pad']
    filter_stride = conv_param['stride']
    input_size = input_dim[1]
    conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride + 1
    pool_output_size = int(fileter_num * (conv_output_size/2) * (conv_output_size/2))

    self.params = {}
    self.params['W1'] = weight_init_std * np.random.randn(filter_num, input_dim[0],filter_size, filter_size)
    self.params['b1'] = np.zeros(filter_num)
    self.params['W2'] = weight_init_std * np.random.randn(pool_output_size, hidden_size)
    self.params['b2'] = np.zeros(hidden_size)
    self.params['W3'] = weight_init_std * np.random.randn(hidden_size, output_size)
    self.params['b3'] = np.zeros(output_size)

    self.layers = {}
    self.layers['Conv1'] = Convolution(self.params['W1'],self.params['b1'], conv_params['stride'], conv_params['pad'])
    self.layers['Relu1'] = Relu()
    self.layers['Pool1'] = Pooling(pool_h= 2, pool_w=2, stride = 2)
    self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
    self.layers['Relu2'] = Relu()
    self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])

    self.last_layer = SoftmaxWightLoss()

  def predict(self,x):
    for layer in self.layers.values():
      x = layer.forward(x)
      return x

  def loss(self,x,t):
    y = self.predict(x)
    return self.lastLayer.forward(y, t)

  def gradient(self, x, t):
    # 순전파
    self.loss(x,t)

    #역전파 
    dout = 1
    dout = self.lastLayer.backward(dout)

    layers = list(self.layers.values())
    layers.reverse()
    for layer in layers:
      dout = layer.backward(dout)


    grads = {}
    grads['W1'] = self.layers['Conv1'].dW
    grads['b1'] = self.layers['Conv1'].db
    grads['W2'] = self.layers['Affine1'].dW
    grads['b2'] = self.layers['Affine1'].db
    grads['W3'] = self.layers['Affine2'].dW
    grads['b3'] = self.layers['Affine2'].db

    return grads

